# Initializing

In [1]:
! pip install Faiss-cpu

In [2]:
import numpy as np
from scipy import spatial
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

In [26]:
np.random.seed(42)

## Helper Function

In [4]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [5]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [6]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [7]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [8]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [9]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [10]:
print(dim)

100


In [11]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

CPU times: user 12 s, sys: 70.9 ms, total: 12.1 s
Wall time: 17.1 s


In [12]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

CPU times: user 1.55 s, sys: 157 ms, total: 1.71 s
Wall time: 1.83 s


In [13]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

CPU times: user 1.28 s, sys: 10.1 ms, total: 1.29 s
Wall time: 1.28 s


In [14]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.138


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed.

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [15]:
# sixth edition

from sklearn import decomposition
from sklearn.cluster import MiniBatchKMeans as MBKM

def brute_search(query_vector, index_vectors, k):
  """
  This function apply brutal search of the top k vectors fitting to the query.
  Args:
      query_vector: A vector containing the query vectors.
      index_vectors: An array of shape (n_index, dim) containing the index vectors.
      k: The amount of vectors to retrieve.
  Returns:
      A closest - array of top k closest vectors to the query.
  """
  # calculate the distances of the vectors to the query
  distances = np.linalg.norm(index_vectors - query_vector, axis=1)

  # take top closest vectors based on distances
  closest = list(np.argsort(distances)[:k])

  return closest

def custom_indexing_algorithm(index_vectors, dim):
  """
  This function builds an index from scratch.
  Args:
      index_vectors: An array of shape (n_index, dim) containing the index vectors.
      dim: The dimensionality of the vectors.
  Returns:
      An index - including the kmeans clustering, the original data and the pca we converted with.
  """
  # transform the data to pca space with dim=2
  pca = decomposition.PCA()
  pca.n_components = 2
  pca_data = pca.fit_transform(index_vectors)

  # cluster the data using MiniBatch Kmeans
  clustering = MBKM(n_clusters=10)
  clustering.fit(pca_data)

  return clustering, index_vectors, pca

def custom_index_search(query_vectors, index, k):
  """
  This function searches over the custom index.
  Args:
      query_vectors: An array of shape (n_queries, dim) containing the query vectors.
      index: The custom index.
      k: The number of nearest neighbors to retrieve.
  Returns:
    An ann_lists - including list with top k vectors fitting to the query, for each query
  """
  # list of lists for the results of the queries
  ann_lists = []

  # extract the indexing from the index we built
  clustering, index_vectors, pca = index

  # find the cluster of the queries
  query_clusters = clustering.predict(pca.transform(query_vectors))

  # for every query apply brutal search on the vectors from her class
  for i, query in enumerate(query_vectors):
    relevant_index = np.array(np.where(np.array(clustering.labels_)==query_clusters[i])[0]) #get the indexes of the vectors from the query's cluster
    ann_lists.append(relevant_index[brute_search(query, index_vectors[relevant_index],k)])  #apply brutal search on all those vectors

  return ann_lists

In [29]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, dim)

CPU times: user 349 ms, sys: 142 ms, total: 490 ms
Wall time: 255 ms


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [31]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

CPU times: user 1.21 s, sys: 2.78 ms, total: 1.21 s
Wall time: 1.2 s


In [32]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")

recall@10 for custom_index_search: 1.0


#Previous Attemps

code of privious attemps. no longer relevant since the latest attemp outperformed them.

In [19]:
# ### first edition

# from sklearn.decomposition import PCA

# def custom_indexing_algorithm(index_vectors, dim):
#   """
#   This function builds an index from scratch.
#   Args:
#       index_vectors: An array of shape (n_index, dim) containing the index vectors.
#       dim: The dimensionality of the vectors.
#   Returns:
#       An index.
#   """

#   hp_num = 20   # constants

#   # contain all the hash tables
#   hash_tables = []

#   # # randomly generate hp_num Hyper Planes
#   # hp = np.random.rand(hp_num, dim)

#   # Use PCA to generate hyperplanes
#   pca = PCA(n_components=hp_num)
#   pca.fit(index_vectors)
#   hp = pca.components_

#   # apply dot product to discover location of indexes
#   dot_products = np.dot(index_vectors, hp.T)

#   # Determine the hash value (1 if dot product > 0, else 0)
#   hash_values = (dot_products > 0).astype(int)

#   hash_table = defaultdict(list)  #start a hash table, Use defaultdict for efficient insertion

#   # Go over the points and assign keys in the hash table
#   for i, hash_val in enumerate(hash_values):
#     hash_key = tuple(hash_val)  # Convert binary hash value to a tuple for use as a dictionary key
#     hash_table[hash_key].append(i)  # Add the index of the point to the hash table at the right key

#     # hash_tables.append((hash_table, hp))
#   return hash_table, hp


# def custom_index_search(query_vectors, index, k):
#   """
#   This function searches over the custom index.
#   Args:
#       query_vectors: An array of shape (n_queries, dim) containing the query vectors.
#       index: The custom index.
#       k: The number of nearest neighbors to retrieve.
#   """
#   # list of lists for the results of the queries
#   ann_lists = []

#   # extract the indexing from the index we built
#   hash_table, hp = index

#   # apply dot product to discover location of queries
#   dot_products = np.dot(query_vectors, hp.T)

#   # Determine the hash value (1 if dot product > 0, else 0)
#   hash_values = (dot_products > 0).astype(int)

#   for i, hash_val in enumerate(hash_values):
#     hash_key = tuple(hash_val)  # Convert binary hash value to a tuple for use as a dictionary key
#     if hash_key in hash_table:
#       ann_lists.append(hash_table[hash_key][:k])
#   return ann_lists

In [20]:
# ### forth edition

# from sklearn.decomposition import PCA

# def brute_search(query_vector, index_vectors, k):
#   print(len(index_vectors))
#   distances = np.linalg.norm(index_vectors - query_vector, axis=1)
#   closest = list(np.argsort(distances)[:k])
#   return closest

# def custom_indexing_algorithm(index_vectors, dim):
#   """
#   This function builds an index from scratch.
#   Args:
#       index_vectors: An array of shape (n_index, dim) containing the index vectors.
#       dim: The dimensionality of the vectors.
#   Returns:
#       An index.
#   """

#   hp_num = 7  # constants

#   # contain all the hash tables
#   hash_tables = []

#   # normaly generate hp_num Hyper Planes
#   hp = np.random.multivariate_normal(np.zeros(dim), np.eye(dim), size=hp_num)

#   # apply dot product to discover location of indexes
#   dot_products = np.dot(index_vectors, hp.T)

#   # Determine the hash value (1 if dot product > 0, else 0)
#   hash_values = (dot_products > 0).astype(int)

#   hash_table = defaultdict(list)  #start a hash table, Use defaultdict for efficient insertion

#   # Go over the points and assign keys in the hash table
#   for i, hash_val in enumerate(hash_values):
#     hash_key = tuple(hash_val)  # Convert binary hash value to a tuple for use as a dictionary key
#     hash_table[hash_key].append(i)  # Add the index of the point to the hash table at the right key

#     # hash_tables.append((hash_table, hp))
#   return hash_table, hp, index_vectors


# def custom_index_search(query_vectors, index, k):
#   """
#   This function searches over the custom index.
#   Args:
#       query_vectors: An array of shape (n_queries, dim) containing the query vectors.
#       index: The custom index.
#       k: The number of nearest neighbors to retrieve.
#   """
#   # list of lists for the results of the queries
#   ann_lists = []

#   # extract the indexing from the index we built
#   hash_table, hp, index_vectors = index

#   # apply dot product to discover location of queries
#   dot_products = np.dot(query_vectors, hp.T)

#   # Determine the hash value (1 if dot product > 0, else 0)
#   hash_values = (dot_products > 0).astype(int)

#   for i, hash_val in enumerate(hash_values):
#     hash_key = tuple(hash_val)  # Convert binary hash value to a tuple for use as a dictionary key
#     if hash_key in hash_table:
#       relevant_index = np.array(hash_table[hash_key])
#       ann_lists.append(relevant_index[brute_search(query_vectors[i], index_vectors[relevant_index],k)])
#   return ann_lists

In [21]:
# ###fifth edition

# def brute_search(query_vector, index_vectors, k):
#   distances = np.linalg.norm(index_vectors - query_vector, axis=1)
#   closest = list(np.argsort(distances)[:k])
#   return closest

# def custom_indexing_algorithm(index_vectors, dim):
#   """
#   This function builds an index from scratch.
#   Args:
#       index_vectors: An array of shape (n_index, dim) containing the index vectors.
#       dim: The dimensionality of the vectors.
#   Returns:
#       An index.
#   """
#   (idx_num, hp_num) = 50, 7     # constants

#   (best_hash_table, best_measure) = defaultdict(list), np.inf    # contain the best hash table

#   #loop for several indexes  ------temporarily start with one hase table.
#   for _ in range(idx_num):

#     # normaly generate hp_num Hyper Planes
#     hp = np.random.multivariate_normal(np.zeros(dim), np.eye(dim), size=hp_num)

#     # apply dot product to discover location of indexes
#     dot_products = np.dot(index_vectors, hp.T)

#     # Determine the hash value (1 if dot product > 0, else 0)
#     hash_values = (dot_products > 0).astype(int)

#     hash_table = defaultdict(list)  #start a hash table, Use defaultdict for efficient insertion

#     # Go over the points and assign keys in the hash table
#     for i, hash_val in enumerate(hash_values):
#       hash_key = tuple(hash_val)  # Convert binary hash value to a tuple for use as a dictionary key
#       hash_table[hash_key].append(i)  # Add the index of the point to the hash table at the right key

#     longest = max(len(item) for item in hash_table.values())

#     if best_measure>(longest):
#       best_hash_table, best_measure = (hash_table, hp), longest

#   return best_hash_table, index_vectors


# def custom_index_search(query_vectors, index, k):
#   """
#   This function searches over the custom index.
#   Args:
#       query_vectors: An array of shape (n_queries, dim) containing the query vectors.
#       index: The custom index.
#       k: The number of nearest neighbors to retrieve.
#   """
#   # list of lists for the results of the queries
#   ann_lists = []

#   # extract the indexing from the index we built
#   (hash_table, hp), index_vectors = index

#   # apply dot product to discover location of queries
#   dot_products = np.dot(query_vectors, hp.T)

#   # Determine the hash value (1 if dot product > 0, else 0)
#   hash_values = (dot_products > 0).astype(int)

#   for i, hash_val in enumerate(hash_values):
#     hash_key = tuple(hash_val)  # Convert binary hash value to a tuple for use as a dictionary key
#     if hash_key in hash_table:
#       relevant_index = np.array(hash_table[hash_key])
#       ann_lists.append(relevant_index[brute_search(query_vectors[i], index_vectors[relevant_index],k)])
#   return ann_lists